This notebook contains functions to extract the following information from a document:
* Keywords
* Top relevant sentences

** Input **
Reads in data stored in pickle format as extracted using Kinshuk's notebook


** Output **
`list_of_keywords`,`top_10_relevant_sentences`


** Usage **
Call `extract_summary` function with the document text as a parameter.


** Notes **


* I tried to use Texttiling in order to tokenize the text by topics and then use it to extract keywords, themes, etc. However, it did not result in any better quality keywords. A new challenge was that of increased number of keywords, hence beating the purpose of summarizing the text. We decided not to use the algorithm in our processing pipeline.
* The function takes into account only unigrams and bigrams while extracting top relevant sentences.

** Imports **

In [1]:
from pickle import dump, load
import nltk
from nltk import word_tokenize,FreqDist
import re
from nltk.corpus import wordnet as wn
from nltk.util import ngrams

In [2]:
def get_document_text(raw_text):
    """ This function takes in raw document text as input which we receive from the API and returns a clean text 
    of the associated document. It cleans up any HTML code in the text, newline characters, and extracts supplemental
    information part of the document.
    
    INPUT: string
    OUTPUT: string
    """
    raw_text = raw_text.replace('\n',' ')
    supp_info_idx = raw_text.find("SUPPLEMENTARY INFORMATION:")
    
    suppl_info = raw_text[supp_info_idx+26:] # To leave out the string 'Supplementary Information'
    # Remove any residual HTML tags in text
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', suppl_info)
    
    return suppl_info

def get_keywords(clean_corpus):
    """ This function takes in a clean corpus as input and extracts most important keywords and top 10% of relevant 
    sentences from the text.
    
    INPUT: string
    OUTPUT: List of tuples: [(list_of_keywords,list_of_sentences)]
    """
    
    tagged_tokens = tag_my_text(tokenize_text_sent(clean_corpus))
    grand_list = get_top_np(noun_phrase_finder(tagged_tokens))
    
    return grand_list

def tokenize_text(corpus):
    pattern = r'''(?x)    # set flag to allow verbose regexps
    (([A-Z]\.)+)       # abbreviations, e.g. B.C.
    |(\w+([-']\w+)*)       # words with optional internal hyphens e.g. after-ages or author's
    '''
    tokens = nltk.regexp_tokenize(corpus,pattern)
    all_token = [word.lower() for token in tokens for word in token if word != "" 
                 and word[0] != "'" and word[0] != "-"]
    return all_token

def tokenize_text_sent(corpus):
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sents = sent_tokenizer.tokenize(corpus) # Split text into sentences    
    return [tokenize_text(sent) for sent in raw_sents]

def tag_my_text(sents):
    return [nltk.pos_tag(sent) for sent in sents]

#Chunk noun phrases in tree 
def noun_phrase_chunker():
    grammar = r"""
    NP: {<DT|PP\$>?<JJ>*<NN>}   # chunk determiner/possessive, adjectives and noun
    """
    cp = nltk.RegexpParser(grammar)
    return cp

#Extract only the NP marked phrases from the parse tree, that is the chunk we defined
def noun_phrase_extractor(sentences, chunker):
    res = []
    for sent in sentences:
        tree = chunker.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'NP' : 
                res.append(subtree[0:len(subtree)])
                #res.append(subtree[0])
                #print(subtree)
    return res

def noun_phrase_finder(tagged_text):
    all_proper_noun = noun_phrase_extractor(tagged_text,noun_phrase_chunker()) 
    #does not literally mean proper noun. Chunker only extracts common noun
    noun_phrase_list = []                                                      
    #noun_phrase_string_list =[]
    for noun_phrase in all_proper_noun:
        if len(noun_phrase) > 0: #this means where the size of the phrase is greater than 1
            small_list =[]
            for (word,tag) in noun_phrase:
                small_list.append(word)
            noun_phrase_list.append(small_list)
            #noun_phrase_string_list.append(' '.join(small_list))
    return noun_phrase_list

#get frequency dist of different length in all the noun phrases extracted. 
#Something of the form {1:45,2:23} - how many 1phrased and 2 phrased chunks I have etc.
def get_length_np(nounPhrase):
    np_length={}
    for inner_np in nounPhrase:
        np_length[len(inner_np)] = np_length.get(len(inner_np),0) + 1
    return np_length

#get freq dist obj for noun phrase of different lengths
def find_freq(nested_list,nest_len):
    #from nltk.probability import FreqDist
    fdist_list =[]
    for inner_np in nested_list:
        if len(inner_np) == nest_len:
            fdist_list.append(' '.join(inner_np))
    fdist = FreqDist(fdist_list)
    return fdist

#make a grand list of top occuring noun phrases of different sizes
def get_top_np(np):
    master_common_list=[]
    len_list =get_length_np(np).keys()
    for item in len_list:
        fdist_np = find_freq(np,item)
        top = fdist_np.most_common(15) 
        top_list = []
        for w,c in top:
            if c >= 10:
#                 print (w)
                top_list.append((w,c))
                #top.remove((w,c))
        if len(top_list) > 0:
            master_common_list.append(top_list)
    return master_common_list

In [37]:
def get_top_sents(corpus,keywords_list):
    sentence_list = get_sentences(corpus)
    indexed_sents = sentence_indexing(sentence_list) # This is so that we can re-order most relevant sentences later
    
    sentence_length_scores = get_sentence_lengths(sentence_list)
    keyphrase_scores = get_keyphrase_scores(corpus,sentence_list)
    
    sent_scores = [s+c for s,c in zip(sentence_length_scores,keyphrase_scores)]
    idx_sent_scores = [(s,c) for s,c in zip(indexed_sents,sent_scores)]
    sorted_sents = sorted(idx_sent_scores,key=lambda sent: sent[1],reverse=True)
    
    # Keep top 10% of the sentences, or top 10 whichever is less
    top_10 = int(len(sorted_sents) * 0.1)
    if top_10 > 10:
        top_10 = 10
    x = sorted_sents[:top_10]
    top_list = [item[0] for item in x]
    sorted_top_list = sorted(top_list,key=lambda sent:sent[1],reverse=False)
    sorted_top_list = [sent[0] for sent in sorted_top_list]
    
    return sorted_top_list
    
def get_sentences(corpus):
    # First, tokenize the corpus into sentences
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sents = sent_tokenizer.tokenize(corpus)
    
    return raw_sents

def sentence_indexing(sent_list):
    indexed_sents = []
    for idx,sent in enumerate(sent_list):
        indexed_sents.append((sent,idx))
    return indexed_sents    

def get_sentence_lengths(sent_list):
    sent_length = []
    for s in sent_list:
        sent_length.append(len(s.split(' ')))
    
    return sent_length

def get_keyphrase_scores(corpus,sent_list):
    keywords = get_keywords(corpus) # This gives us a list containing unigrams at index 0 and bigrams at index 1,etc
    
    unigrams = [item[0] for item in keywords[0]]
    bigrams = [item[0] for item in keywords[1]]

    unigram_scores = get_unigram_scores(unigrams,sent_list)
    bigram_scores = get_bigram_scores(bigrams,sent_list)

    sent_feature_import = [a+b for a,b in zip(unigram_scores,bigram_scores)]
    
    return sent_feature_import

def get_unigram_scores(unigram_list,sent_list):
    occurence_list = []
    for s in sent_list:
        words = s.split(' ')
        occurence_count = 0
        for w in words:
            if w.lower() in unigram_list or w.lower() in ['complaint','concern','documented','evidence','warn']:
                occurence_count += 1
        occurence_list.append(occurence_count)
        
    return occurence_list

def get_bigram_scores(bigram_list,sent_list):
    occurence_list = []
    for s in sent_list:
        # create bigrams
        token=nltk.word_tokenize(s)
        bigram_phrases = ngrams(token,2)
        occurence_count = 0
        for w in bigram_phrases:
            w = [word.lower() for word in w]
            if ' '.join(w) in bigram_list:
                occurence_count += 1
        occurence_list.append(occurence_count)
        
    return occurence_list

In [7]:
def extract_summary(text):
    clean_text = get_document_text(text)
    keywords = get_keywords(clean_text)
    top = get_top_sents(clean_text,keywords)
    
    return keywords,top

** Test **

Testing out the function

In [3]:
# Test
doc_list =load(open("data/Proxima_doc_content",'rb'))
document = doc_list[0]
document_text = []
for item in document['text']:
    document_text.append(str(item))

In [ ]:
keywords,sentences = extract_summary(document_text[0])

** Output **

In [40]:
keywords

[[('trail', 35),
  ('plan', 22),
  ('use', 19),
  ('bicycle', 17),
  ('management', 12),
  ('cuyahoga', 11),
  ('bullet', 10),
  ('executive', 10),
  ('order', 10)],
 [('the park', 29), ('this rule', 16)]]

In [39]:
sentences

["  Background  Legislation and Purposes of Cuyahoga Valley National Park      On December 27, 1974, President Gerald Ford signed Public Law 93- 555 creating Cuyahoga Valley National Recreation Area for the purpose  of ``preserving and protecting for public use and enjoyment, the  historic, scenic, natural, and recreational values of the Cuyahoga  River and the adjacent lands of the Cuyahoga Valley and for the purpose  of providing for the maintenance of needed recreational open space  necessary to the urban environment.''",
 "The goals of the 2009 Plan/EIS were to develop a trail network  that:     <bullet> Provides experiences for a variety of trail users;     <bullet> shares the historic, scenic, natural and recreational  significance of the Park;     <bullet> minimizes impacts to the park's historic, scenic, natural  and recreational resources;     <bullet> can be sustained; and     <bullet> engages cooperative partnerships that contribute to the  success of the Park's trail networ